# Smurf Predictor | Brandon Jackson || Richard Tang || Tuan Quach 
# This ambitious data science experiment's goal is to: given an input (summoner name), return a list of possible smurfs of that account. This project as inspried by an interview I watched with Team Liquid's coach Steve mentioned that when teams are scouting players they often need to get a full list of smurf accounts.
****
# For those of you who do not play the game League of Ledgends, often times a high elo (the rating system for the game) create smurf accounts to practice new champions or get lower queue time (since they must be matched up with players of their skill level as you climb the elo ladder you have fewer players as good as you). You may also want to hide a pocket pick you may bring out in the LCS/LCK/LPL/LMS/etc...(tournment) So there is insentive for coaches to want to know who other players may be practising on as well as recruits they may be scouting and want to get a fuller picture. Out of respect for player's who want to hide their smurf some code that would be avalible to pulbic on github will not be posted. However, if you are a coach or team looking for software like this please contact me at Brandon@devbjax.com(Prefered) || Bjax@cs.umd.edu.

imporvements, find know list of player smurf to get data like off role champoins and builds

In [1]:
!pip install riotwatcher

# Create a list of dataframes continaing games that a summoner has played

In [0]:
# usefull website for idea 
# https://github.com/JixiangFan/CMSC320/blob/master/finalProject/Final_Tutorial_Video_Game1.1-1.ipynb
# https://github.com/umd320Student/umd320Student/blob/master/FinalProject%20(1).ipynb
from riotwatcher import RiotWatcher
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time

def parse_get(url):
  response = requests.request("GET", url)
  return json.loads(response.text)

def get_player(summonerName):
  api_key_raw = 'RGAPI-683d405c-5dfa-424c-87ff-fed7fb7f0b7d'
  api_key = "?api_key=" + str(api_key_raw)
  #Need to generate a new riot API key every 24 hours
  watcher = RiotWatcher(api_key_raw)

  my_region = 'na1'
  #summonerName = 'Doublelift'
  me = watcher.summoner.by_name(my_region, summonerName)
  #print(me)

  account_id = str(me['accountId'])
  #Get list of match ID's by account
  url_matchlists = "https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/" + account_id + api_key
  match_list = parse_get(url_matchlists)

  matches = []
  lanes = []
  for i in range(match_list['endIndex']):
    matches.append(match_list['matches'][i]['gameId'])
    lanes.append(match_list['matches'][i]['lane'])
  ''' a full match look like this
  {'champion': 64,
   'gameId': 2915177882,
   'lane': 'JUNGLE',
   'platformId': 'NA1',
   'queue': 420,
   'role': 'NONE',
   'season': 11,
   'timestamp': 1542855765895}
  '''

  data_list = []
  counter = 0
  for i in matches:
    time.sleep(3) #If this doesn't work increase the wait time as Riot's API has rate limits
    counter = counter + 1
    if(counter == 101):
      break
    #display(i, counter)
    #Get match details from matchID from matchlist
    url_match = "https://na1.api.riotgames.com/lol/match/v4/matches/"+ str(i) + api_key 
    url_timeline = "https://na1.api.riotgames.com/lol/match/v4/timelines/by-match/"+ str(i) + api_key

    #Cross reffrence so that we can figure out who is in timeslines since timeliensv4 does not give summer information
    #However we can use another repsonse to get it using matchesv4
    response = requests.request("GET", url_match)
    parsed_json = json.loads(response.text)
    index = 0
    for j in range(10):
       if summonerName == parsed_json['participantIdentities'][j]['player']['summonerName']:
          index = j
          break

    participantId = parsed_json['participantIdentities'][index]['participantId']
    parsed_json = parse_get(url_timeline)
    #display(parsed_json)
    #Filter dataframe for given summer
    data = json_normalize(parsed_json['frames'])
    list(data)
    filter_list = list(data)
    res = [k for k in filter_list if 'participantFrames.'+str(participantId)+'.' in k]
    res
    data = data[res]
    data_list.append(data)

  return data_list, me

In [0]:
list_of_matches, player = get_player("Doublelift")

In [0]:
list_of_matches2, player2 = get_player("WildTurtle")

In [0]:
def download_as_csv(list):
  from google.colab import files
  counter = 0
  for i in list:
    counter += 1
    i.to_csv('game'+str(counter)+'.csv')
    files.download('game'+str(counter)+'.csv')

### Imports and seed

In [6]:
import numpy as np
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)


Using TensorFlow backend.


### Load CSV since riot has rate limits, it's best to save the data so we can call whenever we want WITHOUT having to run get_player which takes not only forever butis prone to error due to rate limits.

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
print (uploaded['game1.csv'][:200].decode('utf-8') + '...')

### Reason for using certian feutures. While some are obvious, other require more domain knowledge. We keep current gold and total gold as tolal gold and level indicate how well a player is doing. Current gold however indicates how many times a player bases during a game which is highly detemined by a player philsopy and knoledge about the game.

### We want to flatten each dataframe so that we can have one observation be a row instead of being an entire CSV file. This is quit crude but will suffice for this tutorial. We are testing our hypothesis that it is possible to tell players apart by the data given to us by riot games then standardizing it and feeding it to a model.

## May need a different model for x,y potition

In [0]:
# import pandas as pd
import io

# df = pd.read_csv(io.StringIO(uploaded['game1.csv'].decode('utf-8')))
# df
def collapse_into_df(list, player_id):
  #collapse_df = pd.DataFrame(columns=['currentGold', 'jungleMinionsKilled', 'level', 'minionsKilled','position_x', 'position_y','totalGold', 'xp'])
  #print(df.columns)
  rows_list = []
  for df in list:
    #display(df.columns)
    #Will need to add 'Unnamed: 0' to the column assigment below if reading from CSV
    df.columns = ['currentGold', 'dominionScore', 'jungleMinionsKilled', 'level', 'minionsKilled', 'participantId','position_x', 'position_y', 'teamScore','totalGold', 'xp']
    df = df[['currentGold', 'jungleMinionsKilled', 'level', 'minionsKilled','position_x', 'position_y','totalGold', 'xp']]
    dict = {}
    current_Gold_avg = 0
    #Remember these are per frame
    for k in df["currentGold"]:
      current_Gold_avg += k/len(df.index)
    dict['currentGold_per_frame'] = current_Gold_avg
    dict['totalGold_per_frame'] = (df["currentGold"][len(df.index)-1])/(len(df.index)-1)
    dict['jungleMinionsKilled_per_frame'] = (df["jungleMinionsKilled"][len(df.index)-1])/(len(df.index)-1)
    dict['level_per_frame'] = (df["level"][len(df.index)-1])/(len(df.index)-1)
    dict['minionsKilled_per_frame'] = (df["minionsKilled"][len(df.index)-1])/(len(df.index)-1)
    #dict['position_x_per_frame'] = (df["position_x"][len(df.index)-1])/(len(df.index)-1)
    #dict['position_y_per_frame'] = (df["position_y"][len(df.index)-1])/(len(df.index)-1)
    dict['totalGold_per_frame'] = (df["totalGold"][len(df.index)-1])/(len(df.index)-1)
    dict['xp_per_frame'] = (df["xp"][len(df.index)-1])/(len(df.index)-1)
    dict['player'] = player_id
    rows_list.append(dict)
    #print(dict)
  collapse_df = df = pd.DataFrame(rows_list)
  return collapse_df

### Create the model

In [39]:
#collapse_into_df([df]) 
#display(player['accountId'])
data = collapse_into_df(list_of_matches, player['accountId']) #The list of Doublelift's games and his account_id
data

,currentGold_per_frame,jungleMinionsKilled_per_frame,level_per_frame,minionsKilled_per_frame,player,totalGold_per_frame,xp_per_frame
0,627.333333,4.846154,0.500000,0.846154,32971449,360.076923,423.000000
1,684.200000,0.666667,0.541667,5.875000,32971449,408.208333,440.958333
2,1066.400000,0.758621,0.517241,8.000000,32971449,503.448276,473.068966
3,876.217391,0.818182,0.636364,8.136364,32971449,650.590909,549.863636
4,1072.735294,1.333333,0.515152,8.333333,32971449,500.181818,535.030303
5,1066.045455,0.558140,0.418605,7.279070,32971449,526.697674,538.744186
6,498.181818,0.000000,0.468750,2.437500,32971449,316.750000,429.593750
7,525.153846,0.160000,0.520000,8.480000,32971449,371.560000,421.160000
8,1216.454545,0.930233,0.418605,8.348837,32971449,528.627907,529.883721
9,495.925926,0.307692,0.500000,6.692308,32971449,390.576923,400.269231


In [40]:
#collapse_into_df([df]) 
#display(player['accountId'])
data2 = collapse_into_df(list_of_matches2, player2['accountId']) #The list of Doublelift's games and his account_id
data2

,currentGold_per_frame,jungleMinionsKilled_per_frame,level_per_frame,minionsKilled_per_frame,player,totalGold_per_frame,xp_per_frame
0,1319.391304,1.636364,0.681818,8.090909,548787,688.681818,603.136364
1,983.206897,0.714286,0.535714,6.821429,548787,594.464286,502.321429
2,841.821429,0.444444,0.518519,7.296296,548787,446.148148,475.888889
3,809.516129,1.200000,0.566667,6.466667,548787,453.300000,556.600000
4,531.312500,0.387097,0.516129,8.419355,548787,446.032258,483.064516
5,650.869565,0.181818,0.590909,5.636364,548787,442.545455,462.363636
6,846.051282,0.921053,0.473684,8.342105,548787,496.421053,559.315789
7,258.600000,0.000000,0.500000,3.500000,548787,226.250000,129.500000
8,562.333333,0.173913,0.565217,8.043478,548787,400.217391,477.260870
9,524.258065,0.000000,0.500000,6.266667,548787,375.800000,440.500000


In [35]:
df = data.append(data2, ignore_index=True)
df.describe()

,currentGold_per_frame,jungleMinionsKilled_per_frame,level_per_frame,minionsKilled_per_frame,player,totalGold_per_frame,xp_per_frame
count,200.000000,200.000000,200.000000,200.000000,2.000000e+02,200.000000,200.000000
mean,759.761834,0.912424,0.583108,6.511469,1.676012e+07,492.378227,522.649678
std,240.299159,1.349504,0.167838,2.422781,1.625201e+07,168.038275,192.833860
min,140.400000,0.000000,0.375000,0.266667,5.487870e+05,190.500000,115.750000
25%,609.920031,0.197619,0.500000,5.867188,5.487870e+05,391.793478,429.245313
50%,715.867173,0.478095,0.535714,7.132785,1.676012e+07,457.672794,481.934839
75%,867.785455,0.870924,0.583333,8.203571,3.297145e+07,528.672853,534.336406
max,1791.937500,6.520000,1.230769,10.542857,3.297145e+07,1297.692308,1408.125000


In [0]:
# dataframe = data2 #data2 #df this is the full append datframe #pandas.read_csv("game1.csv", header=None)
# dataset = dataframe.values
# # split into input (X) and output (Y) variables
# X = dataset[:,0:7].astype(float)
# dataframe = data['player']
# dataset = dataframe.values
# Y = dataset
#display(Y)
# encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(Y)
# encoded_Y = encoder.transform(Y)

# baseline model
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

def cross_validate(data):
  dataframe = data #data2 #df this is the full append datframe #pandas.read_csv("game1.csv", header=None)
  dataset = dataframe.values
  # split into input (X) and output (Y) variables
  X = dataset[:,0:7].astype(float)
  target = collapse_into_df(list_of_matches, player['accountId']) 
  dataframe = target['player']
  dataset = dataframe.values
  Y = dataset

  # evaluate model with standardized dataset
  estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
  kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
  results = cross_val_score(estimator, X, Y, cv=kfold)
  print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [50]:
data = collapse_into_df(list_of_matches, player['accountId']) 
cross_validate(data)

Results: 100.00% (0.00%)


In [49]:
data2 = collapse_into_df(list_of_matches2, player2['accountId']) 
cross_validate(data2)

Results: 100.00% (0.00%)


### If you only need to know, which champions a summoner plays in general a good approximation would be to use the champion-mastery scores

### /lol/champion-mastery/v3/scores/by-summoner/{summonerId}
****
### For exact numbers, you can get the matchlist entries, as they include the champion id played

### /lol/match/v3/matchlists/by-account/{accountId}

### /lol/spectator/v3/active-games/by-summoner/{summonerId} Get current game information for the given summoner ID

Use http://api.champion.gg/docs/ to compare common builds and find outlier items and build paths

****
### Get the match history then use that match id to get match spefics
Use /lol/match/v3/matchlists/by-account/{accountId}

passed the returned match history
https://na1.api.riotgames.com/lol/match/v3/timelines/by-match/2915177882


# EDA (Explority Data Analysis)

Possible Fun stuff
https://medium.com/pandascore-stories/league-of-legends-getting-champion-coordinates-from-the-minimap-using-deep-learning-48a49d35bb74